# CGAN (Conditional GAN)

cGAN(conditional GANs, cGANs)은 GAN 모델을 확장한 것이다. 
cGAN을 사용하면 특정 조건이나 속성을 지닌 이미지를 생성할 수 있는데 결과적으로 vanlia GAN보다 더 나은 이미지가 생성된다. 



MNIST를 통한 cGAN
- https://github.com/namyouth/cGAN/blob/master/cGAN_MNIST.ipynb



이번 cGAN을 통해 할 것 : 자동으로 얼굴 노화를 수행하는 것.
- cGAN 논문 : https://arxiv.org/pdf/1411.1784.pdf
- 구현하려는 신경망에 대한 논문 (Face aging with cCAN)  : https://arxiv.org/pdf/1702.01983.pdf

cGAN에서 다룰 내용
- 얼굴 노화 처리용 GAN 소개
- 프로젝트 구성
- 데이터 준비
- cGAN을 케라스로 구현
- cGAN 훈련
- evaluation 과 hypterparameter조율
- 실용적인 얼굴 노화 어플리케이션

# 얼굴 노화 처리용 GAN 소개
지금까지 우리는 각 용도별로 서로 다른 GAN 신경망들을 구현해 보았다. cGAN은 vanila GAN이라는 개념을 확장한 것으로 generator 신경망의 출력을 제어할 수 있다. 얼굴 노화란 사람의 정체성을 바꾸는 일 없이 그 사람의 얼굴 나이를 바꾸는 일을 말한다. 대부분의 다른 모델들에서는 얼굴 표정이나 선글라스나 턱수염과 같은 얼굴 장식을 고려하지 않으므로 사람의 외모나 정체성에 관한 정보 중에 50% 정도를 잃게 된다. Age-cGAN은 이러한 특성(attributes)들을 모두 고려한다.

## cGAN 이해하기
cGAN은 일부 추가 정보 (extra information)을 조건으로 삼는 GAN 유형이다. 우리는 추가 정보 y를 추가 입력 계층 형태로 generator에 공급한다. vanila GAN에서는 생성된 이미지들의 범주를 전혀 제어하지 않는다. generator에 조건 y를 추가하면 우리는 클래스 레이블이나 정수 데이터와 같은 어떤 데이터 종류일 수 있는 y를 사용해 특정 범주(categories)에 맞는 이미지를 생성할 수 있다. vanila GAN은 하나의 범주만 사용할 수 있으므로 여러 범주에 맞춰 GAN들은 설계하기 무척 어렵다. 


그러나 cGAN을 사용하면 범주별로 조건이 다른 multi-modal models, 즉 다중 최빈값 모델을 생성할 수 있다.

먼저 기존 GAN의 공식을 알아보자.

### 1-1. GAN Network Design

<img src='img/gan_design.png'>



### 1-2. GAN Cost Function
<img src='img/gan_cost.png'>

진짜 이미지와 비슷한 이미지를 생성하는 generator 신경망을 만들기 위해서는 우리는 generator가 생성된 데이터와 진짜 데이터 간의 유사도를 높여야 한다. 그러므로 object function을 사용하여 유사도를 측정한다. 



훈련 도중 D는 총 출력을 최대화하기를 바라지만 G는 최소화하기 바란다. 그래서 GAN을 훈련하게 되면 이 생성기 신경망과 판별기 신경망이 균형이 잡히게 된다. 여기서 GAN이 균형을 잡게 되었을 때 우리는 모델이 수렴했다고 한다. 이 균형은 내쉬 균형이다.


<img src = 'img/gan_cost.png'>

### 1-3. Training GAN
1. D를 'trainable'로 설정한다.
2. 진짜 이미지와 G가 생성한 이미지를 D에 학습시킨다.
3. D를 'non-trainable'로 설정한다.
4. GAN의 일부분으로 G를 훈련시킨다. latent sample을 GAN에 공급하고 G가 이미지를 생성하고 D를 이용하여 이미지를 분류하게 한다. 

그렇다면 cGAN을 알아보자.

### 2-1. cGAN Network Design


<img src='img/cgan_img.png'>

### 2-2. cGAN Cost Function


<img src='img/cgan_cost.png'>



GAN에 비해 cGAN이 다른점은 의외로 간단하다. G와 D에 특정 condition을 나타내는 정보 y를 가해주면 된다.

여기서 y의 형태는 특별하게 정해진 것이 아니기 때문에 다양한 형태를 가질 수 있다. 예를 들어, MNIST에서 원하는 숫자를 generator하고 싶다면 숫자의 class에 해당하는 label(one-hot-encoding)을 추가로 넣어주면 된다. 만약 MNIST를 one-hot-encoding을 하면 10bit가 필요하기 때문에 y는 10bit가 된다. 이뿐만 아니라 다양한 형태도 가능한 mutil modal condition 또한 가능하다.

### 2-3. Networks of cGAN
아키텍처를 살펴보면 Age-cGAN은 네 개의 신경망으로 구성되는데 Encoder, FaceNet, Generator, Discriminator로 구성된다.

1. Encoder의 사용: 입력 이미지들의 역사상(inverse mapping)과 잠재적 벡터를 사용하는 나이 조건을 학습
2. FaceNet의 사용 : 얼굴 인식 신경망으로 입력 이미지 x개와 재구성한 이미지인 x'간의 차이를 학습
3. Generator의 사용 : 얼굴 이미지와 상태 벡터로 구성된 은닉 표현을 취하여 이미지를 생성
4. Discriminator의 사용 : 진짜 이미지와 가짜 이미지를 판별해 내기위한 것


cGAN의 문제점 → 속성 y가 있는 입력 이미지 x를 잠재 벡터 z에 inverse mapping하는 작업을 할 수 없다는 점이다. 이때 encoder 신경망을 사용하는 것이 이 문제를 해결하는 방법이다. 우리는 입력이미지에 해당하는 x의 inverse mapping을 근사하기 위해 encoder 신경망을 훈련할 수 있다. 위 4개의 신경망에 대해 좀더 자세히 알아보자.

### 1. Encoder
공급 받은 이미지들의 잠재 벡터를 생성하는 게 encoder 신경망의 첫 목표이다. encoder는 기본적으로 (64, 64, 3)차원으로 된 이미지 한 장을 취해 100차원의 벡터로 바꾼다. encoder는 심층 합성곱 신경망이다. 이 신경망에는 네 개의 conv block과 두 개의 dense layers로 구성된다. 각 conv block에는 conv layer 한 개, batchnorm layer 한 개, activation function 한 개를 담는다. 각 conv block에서 각 conv layer는 첫 번째 conv layer을 제외하고는 batchnorm layer가 따라 온다. 


### 2. Generator
G의 주요 목표는 (64, 64, 3) 차원으로 된 이미지를 생성하는 데 있다. G는 100차원으로 된 잠재 벡터와 일부 부가 정보인 y를 취해 진짜 같은 이미지들을 생성하려고 시도한다. G 또한 심층 합성곱 신경망이다. Dense layer과 상향 표본 추출 계층 및 conv layer으로 구성된다. 잡음 벡터와 conditioning value라는 두가지 값을 취한다. conditioning value는 신경망에 공급하는 부가 정보를 의미한다. Age-cGAN의 경우는 나이가 이에 해당한다. 


### 3. Discriminator
공급된 이미지가 진짜인지 아닌지 식별해내는 것이 D의 핵심 목표이다. D는 해당 이미지를 연달아 있는 하향 표본추출 계층들과 일부 분류 계층들을 거치게 전달하는 식으로 이미지를 식별해낸다. 다시 말해 D는 이미지가 진짜인지 아니면 가짜인지를 예측하는 것이다. 이 또한 D도 심층 합성곱 신경망이다. D에서는 여러 개의 conv block이 있다. 각 conv block에는 conv layer 한 개, batchnorm 한 개, activation function 한 개가 들어있지만 G처럼 첫 번째 conv block에는 예외적으로 batchnorm layer가 없다.


### 4. FaceNet
주어진 이미지를 바탕으로 특정인의 정체성(identity)을 인식하는 것이 face recognition의 핵심이다. 우리의 과정에서는 pre-trained Inception-ResNet-2 모델을 사용하고 이 모델에는 Fully connected layer가 없다. 이 모델에 일단 이미지 한개를 공급하면 이에 대응하는 임베딩을 돌려준다. 진짜 이미지와 재구성 이미지에서 추출한 임베딩을 계산할 때는 임베딩의 유클리드 거리를 계산한다. (임베딩 및 유클리드 거리는 다음장에 설명)

---

# Age-cGAN의 훈련 단계

1. cGAN 훈련 : G와 D를 훈련한다.
2. 초기 잠재 벡터 근사(Initial latent vector approximation) : encoder를 훈련한다.
3. 잠재 벡터 최적화(latent vector optimization) : encoder 및 generator를 모두 최적화 한다.

<img src='img/age-cgan.png'>
출처 : https://arxiv.org/pdf/1702.01983.pdf

## 1. cGAN 훈련
이 단계에서는 G와 D를 훈련한다. 훈련을 하고 나면 G는 흐릿하게 처리한 얼굴 이미지를 생성할 수 있다. 이 단계에서 vanila GAN 훈련 때와 비슷하게 훈련을 진행한다.

### Object function
cGAN에서는 아래와 같은 object function을 사용할 수 있다.

<img src='img/cgan.png'>

cGAN 을 훈련하는 일은 함수 v(θG, θD)를 최적화하는 일과 관련이 있다. cGAN 훈련을 MinMax 게임으로 생각할 수 있다. 여기서 log D(x, y)는 D의 손실이고 [log (1 − D(G(z, ye), ye))는  G의 손실이며 Pdata는  도는 있음직한 이미지들의 분포이다.


## 2. 초기 잠재 벡터 근사 (Initial latent vector approximation)

초기 잠재 벡터 근사란 얼굴 이미지 재구성 작업을 최적화하기 위해 잠재 벡터에 근사하기 위한 방법이다


cGAN은 이미지에서 잠재 벡터로의 reverse mapping 을 배우지 않는다. 
대신 인코더는 이 reverse mapping을 배우고 목표 연령에서 얼굴 이미지를 생성하는 데 사용할 수있는 latent vector를 생성 할 수 있다. 

우리는 생성된 이미지와 진짜 이미지를 가지고 encoder를 훈련한다. 훈련을 마치고 나면 encoder는 학습한 분포에서 잠재 벡터(latent vector)들을 생성해내기 시작한다. encoder 를 훈련하기 위한 object function 훈련은 유클리드 거리 손실(Euclidean distance loss) 에 해당한다.


이것에 대한 내용으로 인코더 네트워크를 빌드하고 컴파일하려면 다음 코드를 추가해야 한다.

In [4]:
Build Encoderencoder = build_encoder()encoder.compile(loss=euclidean_distance_loss, optimizer='adam')

### 유클리드 거리 손실(Euclidean distance loss)란?
유클리드 거리는 단순히 두 점에서의 거리가 아니라 두 확률변수에서 정의된 거리이다. 예를 들어 (x,y) 가 n 개일 때, n – 차원 공간 상에서 두 점 (x1,x2,⋯,xn) 과 (y1,y2,⋯,yn) 의 거리이다.
즉, 유클리드 거리 손실이란 이 거리를 최소화 하는 것이다.

<img src = 'img/euclidean.png'>


여기서는 녹색과 같은 최단거리가 유클리드 거리이다


## 3. 잠재 벡터 최적화 (latent vector optimization)

이 프로젝트의 궁극적 목표인 얼굴 노화 작업은 사람의 나이를 변경해야하지만 자신의 정체성은 그대로 유지되어야한다고 가정한다.

ground truth 이미지 x와 재구성된 이미지 x' 사이의 픽셀 단위 유클리드 거리를 최소화하기 위해 잠재 벡터 z를 최적화함으로써 이미지 재구성 향상의 유사한 문제가 해결된다.

그러나 여기서 Pixelwise latent vector 최적화에는 두 가지 단점이 있다.
1. 재구성의 흐릿함을 증가시킨다는 점
2. 입력 얼굴 이미지의 불필요한 세부 사항에 중점을 둔다는 점 (선글라스, 헤어스타일 등)

이를 해결하기 위해 'Identity Preserving' latent vector 최적화 방법을 사용한다. 이것은 입력 이미지 x에서 사람의 신원을 인식할 수 있는 얼굴 인식 신경망 FP을 고려했을 때, 오리지널과 재구성된 이미지에서 신원들의 차이점이 유클리드 거리로 표현될 수 있다. 이 유클리드 거리는 일치하는 임베딩 FR(x)와 FP(x') 사이이다.


즉, encodeing은 FR(x)이고 generator는 FP(x')이다.


이렇듯 잠재 벡터 최적화를 하는 동안에 우리는 encoder와 generator를 동시에 최적화한다. 잠재 벡터 최적화를 위해 사용하는 방정식은 아래에 해당한다.

<img src='img/optimization.png'>
          
FR은 Face Recognition 신경망이다. 이 방정식은 진짜 이미지와 재구성한 이미지들간의 유클리드 거리를 가르키는데 이 유클리드 거리가 최소가 되어야 한다. 이번 단계에서 우리는 해당 거리를 최소가 되게 함으로써 정체성 보존을 최대화하려고 노력할 것이다.

### Reference
- http://blog.naver.com/PostView.nhn?blogId=laonple&logNo=221306150417
- 실전! GAN 프로젝트
- https://github.com/mafda/generative_adversarial_networks_101/blob/master/src/mnist/03_CGAN_MNIST.ipynb
- https://github.com/mafda/generative_adversarial_networks_101/blob/master/src/mnist/01_GAN_MNIST.ipynb
- https://www.arxiv-vanity.com/papers/1702.01983/